# 비동기 요청

aiohttp는 파이썬의 asyncio 라이브러리를 기반으로 하는 비동기 HTTP 클라이언트 및 서버 프레임워크이다.  
단일 스레드 내에서 이벤트 루프를 통해 다수의 네트워크 요청을 병렬로 처리하여 입출력 대기 시간을 효율적으로 관리한다.

`pip install aiohttp`를 통해 설치한다.

In [17]:
import asyncio
import aiohttp

async def main():
    # ClientSession은 연결 풀을 관리하는 객체이다.
    async with aiohttp.ClientSession() as session:
        # 비동기 context manager를 사용하여 세션을 생성한다.
        pass


await main()

### 비동기 클라이언트 요청

클라이언트 세션을 생성한 후 get, post 등 HTTP 메서드에 대응하는 함수를 호출하여 요청을 보낸다.  
요청 시 await 키워드를 사용하여 응답이 올 때까지 제어권을 이벤트 루프에 반환한다.

In [ ]:
async def fetch_status(url):
    async with aiohttp.ClientSession() as session:
        # GET 요청을 비동기적으로 수행한다.
        async with session.get(url) as response:
            # 응답 상태 코드를 반환받는다.
            status = response.status
            print(f"상태 코드: {status}")
            return status

await fetch_status("https://jsonplaceholder.typicode.com/posts")

상태 코드: 200


200

### 응답 데이터 처리

응답 객체로부터 텍스트, 바이너리, 제이슨 데이터를 비동기적으로 추출한다.  
데이터 추출 메서드인 text(), json(), read() 역시 코루틴이므로 await 키워드를 사용해야 한다.

In [4]:
async def get_json_data():
    url = "https://jsonplaceholder.typicode.com/posts/1"
    async with aiohttp.ClientSession() as session:
        async with session.get(url) as response:
            # 응답 본문을 제이슨 형식으로 파싱한다.
            data = await response.json()
            # 특정 키의 값을 출력한다.
            print(f"제목: {data['title']}")

await get_json_data()

제목: sunt aut facere repellat provident occaecati excepturi optio reprehenderit


### 다중 요청 병렬 처리

asyncio.gather를 사용하여 여러 개의 aiohttp 요청 태스크를 동시에 실행한다.  
동기 방식의 요청과 달리 각 요청의 완료를 기다리지 않고 다음 요청을 바로 수행한다.

In [5]:
async def fetch_url(session, url):
    async with session.get(url) as response:
        return await response.text()

async def request_all(urls):
    async with aiohttp.ClientSession() as session:
        # 각 URL에 대한 요청 코루틴 리스트를 생성한다.
        tasks = [fetch_url(session, url) for url in urls]
        # 모든 태스크를 병렬로 실행하고 결과를 수집한다.
        responses = await asyncio.gather(*tasks)
        print(f"응답 개수: {len(responses)}")

url_list = [
    "https://jsonplaceholder.typicode.com/posts/1", 
    "https://jsonplaceholder.typicode.com/posts/2", 
    "https://jsonplaceholder.typicode.com/posts/3", 
    ]
await request_all(url_list)

응답 개수: 3


### 매개변수 및 헤더 전달

요청 시 params 인자를 통해 쿼리 스트링을 전달하거나, headers 인자를 통해 HTTP 헤더를 설정한다.  
딕셔너리 구조를 활용하여 데이터를 정의한다.

In [6]:
async def search_with_params():
    url = "https://httpbin.org/get"
    # 쿼리 매개변수를 정의한다.
    query_params = {"name": "admin", "id": "123"}
    # 사용자 정의 헤더를 정의한다.
    custom_headers = {"User-Agent": "AiohttpClient/1.0"}

    async with aiohttp.ClientSession() as session:
        async with session.get(url, params=query_params, headers=custom_headers) as response:
            result = await response.json()
            # 서버에서 수신한 인자 정보를 확인한다.
            print(result["args"])

await search_with_params()

{'id': '123', 'name': 'admin'}


### 다중 요청 개수 제한

비동기 환경에서 동시에 너무 많은 네트워크 요청을 보낼 경우 서버 측 차단이나 시스템 자원 고갈이 발생할 수 있다.  
asyncio.Semaphore 객체를 사용하여 동시에 실행 가능한 코루틴의 숫자를 제어한다.

In [12]:
import asyncio
import aiohttp

async def fetch_with_semaphore(semaphore, session, url):
    # 세마포어를 사용하여 동시 실행 숫자를 제한한다.
    async with semaphore:
        async with session.get(url) as response:
            status = response.status
            # 실제 요청이 수행되는 시점을 확인한다.
            print(f"요청 완료: {url} (상태: {status})")
            return await response.text()

async def main():
    urls = [f"https://jsonplaceholder.typicode.com/posts/{i}" for i in range(1, 21)]
    
    # 동시 요청 숫자를 5개로 제한하는 세마포어를 생성한다.
    semaphore = asyncio.Semaphore(5)
    
    async with aiohttp.ClientSession() as session:
        # 모든 요청 태스크를 생성하되, 세마포어에 의해 5개씩 순차적으로 실행된다.
        tasks = [fetch_with_semaphore(semaphore, session, url) for url in urls]
        await asyncio.gather(*tasks)

await main()

요청 완료: https://jsonplaceholder.typicode.com/posts/1 (상태: 200)
요청 완료: https://jsonplaceholder.typicode.com/posts/5 (상태: 200)
요청 완료: https://jsonplaceholder.typicode.com/posts/3 (상태: 200)
요청 완료: https://jsonplaceholder.typicode.com/posts/4 (상태: 200)
요청 완료: https://jsonplaceholder.typicode.com/posts/2 (상태: 200)
요청 완료: https://jsonplaceholder.typicode.com/posts/6 (상태: 200)
요청 완료: https://jsonplaceholder.typicode.com/posts/7 (상태: 200)
요청 완료: https://jsonplaceholder.typicode.com/posts/8 (상태: 200)
요청 완료: https://jsonplaceholder.typicode.com/posts/9 (상태: 200)
요청 완료: https://jsonplaceholder.typicode.com/posts/10 (상태: 200)
요청 완료: https://jsonplaceholder.typicode.com/posts/11 (상태: 200)
요청 완료: https://jsonplaceholder.typicode.com/posts/12 (상태: 200)
요청 완료: https://jsonplaceholder.typicode.com/posts/13 (상태: 200)
요청 완료: https://jsonplaceholder.typicode.com/posts/14 (상태: 200)
요청 완료: https://jsonplaceholder.typicode.com/posts/15 (상태: 200)
요청 완료: https://jsonplaceholder.typicode.com/posts/16 (상태: 200)
요

### 문제
TMDB API를 활용하여 현재 상영 중인 영화(now_playing) 데이터에서 다음 정보만 담긴 리스트를 만드세요.

- title (영화 제목)  
- vote_average (평점)  
- revenue (수익)

In [5]:
# 1. 사용하던 now_playing을 활용해 데이터 20개 가져오기(일반 동기 써도 무방)
# 2. movie_id 모아놓기
# 3. 그걸 바탕으로 movie/{movie_id}에 요청 동시에 보내기
import requests
from pprint import pprint
import os
from dotenv import load_dotenv


load_dotenv()

# 로드된 환경변수 참조
debug_mode = os.getenv('DEBUG')
secret_key = os.getenv('SECRET_KEY')
server_port = os.getenv('PORT')
TMDB_API = os.getenv('TMDB_API_KEY')

URL = "https://api.themoviedb.org/3/movie/now_playing"
API_KEY = TMDB_API

params = {
    'language' : 'ko-kr',
    # 'api_key' : "짧은 그 키"
}

headers = {
    "Authorization" : f"Bearer {API_KEY}"
}

try:
    response = requests.get(URL, headers=headers, params=params)
    response.raise_for_status()
    data = response.json()
    data = data['results']

    movieid_list=[]
    result=[]

    for movie in data:
        title = movie['title']
        vote_average = movie['vote_average']
        movieid = movie['id']
        movieid_list.append(movieid)
    print(movieid_list)

    for movie_id in movieid_list:
        URL = f"https://api.themoviedb.org/3/movie/{movie_id}"
        response = requests.get(URL, headers=headers, params=params)
        response.raise_for_status()
        data = response.json()
        title=data['title']
        vote_average=data['vote_average']
        revenue=data['revenue']
        result.append(f"제목 : {title}, 평점 : {vote_average} 수익 : {revenue}")
    
    


    pprint(result)



except Exception as e:
    print(e)



[1306368, 83533, 1368166, 1272837, 1419406, 1131759, 1208348, 1228246, 1034716, 1282440, 1491902, 1054867, 1247002, 1223601, 840464, 1317288, 1234731, 1539104, 639988, 798645]
['제목 : 더 립, 평점 : 7.164 수익 : 0',
 '제목 : 아바타: 불과 재, 평점 : 7.355 수익 : 1320000000',
 '제목 : 하우스메이드, 평점 : 7.189 수익 : 245700000',
 '제목 : 28년 후: 뼈의 사원, 평점 : 7.0 수익 : 3120000',
 '제목 : 포풍추영, 평점 : 7.16 수익 : 174400000',
 '제목 : 전지적 독자 시점, 평점 : 6.7 수익 : 9187679',
 '제목 : 렌탈 패밀리: 가족을 빌려드립니다, 평점 : 7.941 수익 : 10706787',
 '제목 : 프레디의 피자가게 2, 평점 : 6.766 수익 : 236746717',
 '제목 : 우리의 열 번째 여름, 평점 : 7.091 수익 : 0',
 '제목 : हैप्पी पटेल: खतरानक जासूस, 평점 : 7.8 수익 : 0',
 '제목 : ปัง, 평점 : 6.489 수익 : 0',
 '제목 : 원 배틀 애프터 어나더, 평점 : 7.433 수익 : 206311045',
 '제목 : 파과, 평점 : 6.5 수익 : 0',
 '제목 : 시수: 복수의 길, 평점 : 7.429 수익 : 9724644',
 '제목 : 그린랜드 2, 평점 : 6.627 수익 : 11416907',
 '제목 : 마티 슈프림, 평점 : 8.082 수익 : 92925093',
 '제목 : 아나콘다, 평점 : 5.9 수익 : 112467421',
 '제목 : 극장판 주술회전: 시부야사변 X 사멸회유, 평점 : 5.628 수익 : 44559195',
 '제목 : 어쩔수가없다, 평점 : 7.75 수익 : 25343108',
 '제목 

In [6]:
# 1. 사용하던 now_playing을 활용해 데이터 20개 가져오기(일반 동기 써도 무방)
# 2. movie_id 모아놓기
# 3. 그걸 바탕으로 movie/{movie_id}에 요청 동시에 보내기
import requests
from pprint import pprint
import os
from dotenv import load_dotenv
import asyncio
import aiohttp


load_dotenv()

# 로드된 환경변수 참조
debug_mode = os.getenv('DEBUG')
secret_key = os.getenv('SECRET_KEY')
server_port = os.getenv('PORT')
TMDB_API = os.getenv('TMDB_API_KEY')

URL = "https://api.themoviedb.org/3/movie/now_playing"
API_KEY = TMDB_API

params = {
    'language' : 'ko-kr',
    # 'api_key' : "짧은 그 키"
}

headers = {
    "Authorization" : f"Bearer {API_KEY}"
}

try:
    response = requests.get(URL, headers=headers, params=params)
    response.raise_for_status()
    data = response.json()
    data = data['results']

    movieid_list=[]
    result=[]

    for movie in data:
        title = movie['title']
        vote_average = movie['vote_average']
        movieid = movie['id']
        movieid_list.append(movieid)
    print(movieid_list)
    
    async def fetch_movie_detail(session,movie_id):
        URL = f"https://api.themoviedb.org/3/movie/{movie_id}"
        async with session.get(URL,headers=headers, params=params) as response:
            data = await response.json()
            title = data.get('title')
            vote_average=data.get('vote_average')
            revenue = data.get('revenue')

            return f"제목: {title}, 평점: {vote_average}, 수익: {revenue}"
        
    async def main():
        async with aiohttp.ClientSession() as session:
            tasks=[]
            for movie_id in movieid_list:
                tasks.append(fetch_movie_detail(session,movie_id))
            final_results = await asyncio.gather(*tasks)
            pprint(final_results)
    
    # asyncio.run(main())
    await main()
    

except Exception as e:
    print(e)



[1306368, 83533, 1368166, 1272837, 1419406, 1131759, 1208348, 1228246, 1034716, 1282440, 1491902, 1054867, 1247002, 1223601, 840464, 1317288, 1234731, 1539104, 639988, 798645]
['제목: 더 립, 평점: 7.164, 수익: 0',
 '제목: 아바타: 불과 재, 평점: 7.355, 수익: 1320000000',
 '제목: 하우스메이드, 평점: 7.189, 수익: 245700000',
 '제목: 28년 후: 뼈의 사원, 평점: 7.0, 수익: 3120000',
 '제목: 포풍추영, 평점: 7.16, 수익: 174400000',
 '제목: 전지적 독자 시점, 평점: 6.7, 수익: 9187679',
 '제목: 렌탈 패밀리: 가족을 빌려드립니다, 평점: 7.941, 수익: 10706787',
 '제목: 프레디의 피자가게 2, 평점: 6.766, 수익: 236746717',
 '제목: 우리의 열 번째 여름, 평점: 7.091, 수익: 0',
 '제목: हैप्पी पटेल: खतरानक जासूस, 평점: 7.8, 수익: 0',
 '제목: ปัง, 평점: 6.489, 수익: 0',
 '제목: 원 배틀 애프터 어나더, 평점: 7.433, 수익: 206311045',
 '제목: 파과, 평점: 6.5, 수익: 0',
 '제목: 시수: 복수의 길, 평점: 7.429, 수익: 9724644',
 '제목: 그린랜드 2, 평점: 6.627, 수익: 11416907',
 '제목: 마티 슈프림, 평점: 8.082, 수익: 92925093',
 '제목: 아나콘다, 평점: 5.9, 수익: 112467421',
 '제목: 극장판 주술회전: 시부야사변 X 사멸회유, 평점: 5.628, 수익: 44559195',
 '제목: 어쩔수가없다, 평점: 7.75, 수익: 25343108',
 '제목: 더 러닝 맨, 평점: 6.8, 수익: 68615641']
